# Llama-index Examples

- [ドキュメント](https://docs.llamaindex.ai/en/stable/examples/query_engine/knowledge_graph_query_engine/?h=knowl)の内容を実行する
- [Nebula Graph](https://www.nebula-graph.io/)の使い方を調べる
    - こちらの[ドキュメント](https://docs.nebula-graph.io/3.6.0/2.quick-start/1.quick-start-workflow/)に従って `Docker Compose` を活用して実行
        - `docker exec -it nebula-docker-compose-console-1 /bin/bash` のNebula Consoleの操作方法
        - `/ # ./usr/local/bin/nebula-console -u root -p root --address=graphd --port=9669` の接続
    - Nebula Graphのコマンドはこちらの[ドキュメント](https://docs.nebula-graph.io/3.8.0/3.ngql-guide/9.space-statements/1.create-space/)を参照
    
    - llamaindexのスペースの作成手順

        ```bash
        ❯ docker exec -it nebula-docker-compose-console-1 /bin/sh
        / # ./usr/local/bin/nebula-console -u root -p root --address=graphd --port=9669
                                                                                                                                                                                                    Welcome!

        (root@nebula) [(none)]> CREATE SPACE llamaindex(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1)
        Execution succeeded (time spent 17.518ms/18.580547ms)

        Fri, 12 Jul 2024 07:31:35 UTC

        (root@nebula) [(none)]> :sleep 10
        (root@nebula) [(none)]> USE llamaindex
        Execution succeeded (time spent 1.444ms/2.040943ms)

        Fri, 12 Jul 2024 07:31:56 UTC

        (root@nebula) [llamaindex]> CREATE TAG entity(name string)
        Execution succeeded (time spent 14.981ms/15.586641ms)

        Fri, 12 Jul 2024 07:32:04 UTC

        (root@nebula) [llamaindex]> CREATE EDGE relationship(relationship string)
        Execution succeeded (time spent 2.699ms/3.224067ms)

        Fri, 12 Jul 2024 07:32:10 UTC

        (root@nebula) [llamaindex]> :sleep 10
        (root@nebula) [llamaindex]> CREATE TAG INDEX entity_index ON entity(name(256))
        Execution succeeded (time spent 3.516ms/4.170609ms)

        Fri, 12 Jul 2024 07:32:28 UTC

        (root@nebula) [llamaindex]>    
        ```

In [37]:
# Loggingの設定

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [38]:
# OpenAIの作成

from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.chunk_size = 512

In [39]:
# Nebula Graphのコンフィグ

import os

os.environ["NEBULA_USER"] = "root"
os.environ["NEBULA_PASSWORD"] = "nebula"  # default is "nebula"
os.environ["NEBULA_ADDRESS"] = (
    "127.0.0.1:9669"  # assumed we have NebulaGraph installed locally
)

space_name = "llamaindex"
edge_types, rel_prop_names = ["relationship"], [
    "relationship"
]  # default, could be omit if create from an empty kg
tags = ["entity"]  # default, could be omit if create from an empty kg

In [40]:
# NebulaGraphの接続
# 「llamaindexのスペースの作成手順」を実施後に実行する

from llama_index.core import StorageContext
from llama_index.graph_stores.nebula import NebulaGraphStore

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

## Knowledge Graphの構築手順

1. `WikipediaReader` を使って情報を取得する
1. `Document` に切り替える
1. `KnowledgeGraphIndex` を生成する

※ 構築に2分ぐらいかかる

In [5]:
# Documentのサンプル情報を取得する

from llama_index.readers.wikipedia import WikipediaReader

loader = WikipediaReader()
documents = loader.load_data(
    pages=["Guardians of the Galaxy Vol. 3"], auto_suggest=False, lang_prefix="en"
)

In [41]:
# KnowledgeGraphIndexを構築する

from llama_index.core import KnowledgeGraphIndex

kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    # space_name=space_name,
    # edge_types=edge_types,
    # rel_prop_names=rel_prop_names,
    # tags=tags,
    include_embeddings=True,
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST

In [7]:
# Nebula Graphに接続してSpaceの情報を取得する
%load_ext ngql
%ngql --address 127.0.0.1 --port 9669 --user root --password root

Connection Pool Created
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,Name
0,llamaindex


In [8]:
# llamaindexの検索を行う

%ngql USE llamaindex;
%ngql MATCH ()-[e]->() RETURN e LIMIT 10

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,e
0,"(""Adam"")-[:relationship@9005598024634048093{relationship: ""Stabbed by""}]->(""Nebula"")"
1,"(""Adam mckay"")-[:relationship@-7875718538987686817{relationship: ""Said he was willing to work with""}]->(""Marvel studios again"")"
2,"(""Adam warlock"")-[:relationship@-7355033301802044942{relationship: ""Created by""}]->(""High priestess ayesha"")"
3,"(""Adam warlock"")-[:relationship@-6025549415605064876{relationship: ""Importance on""}]->(""Cosmic side"")"
4,"(""Adam warlock"")-[:relationship@4003220439395659920{relationship: ""Created by""}]->(""High priestess ayesha"")"
5,"(""Adam warlock"")-[:relationship@7673960981714070136{relationship: ""Noted""}]->(""Importance"")"
6,"(""Alan f. horn"")-[:relationship@2023067982055303897{relationship: ""Stated""}]->(""Offensive attitudes"")"
7,"(""Alan f. horn"")-[:relationship@7888911368531140309{relationship: ""Stated""}]->(""Offensive attitudes"")"
8,"(""Allmovie"")-[:relationship@-9163102535676806896{relationship: ""At""}]->(""Guardians of the galaxy vol. 3"")"
9,"(""Allmovie"")-[:relationship@2569255774608074042{relationship: ""At""}]->(""Guardians of the galaxy vol. 3"")"


In [9]:
# draw the result
# 結果が出力されない
%ng_draw

<class 'pyvis.network.Network'> |N|=12 |E|=10

## サンプルコードでは以下のエラーが発生

```bash
ValidationError: 1 validation error for LLMPredictStartEvent
template
  none is not an allowed value (type=type_error.none.not_allowed)
```

- [issue#13741](https://github.com/run-llama/llama_index/issues/13741)より、promptを定義することで回避できる

## 上記対策を行っても以下のエラーが発生する
   
``` bash
ValueError: Query failed. Query: Question: What are the top 5 most influential nodes in the knowledge graph based on their number of relationships?

Cypher Query:

MATCH (n)
RETURN n, size((n)--()) AS relationships
ORDER BY relationships DESC
LIMIT 5, Param: {}Error message: SyntaxError: syntax error near `Question'

The above exception was the direct cause of the following exception:

RetryError                                Traceback (most recent call last)
RetryError: RetryError[<Future at 0x7ff08493c890 state=finished raised ValueError>]"
```
   
## 作成するPromptをNebula Graph用にLLMに書き換えてもらったがダメ
   
```bash
{
ValueError: Query failed. Query: MATCH (p:Person {name: 'John Doe'})-[:FRIENDS_WITH]->(friend)
RETURN friend.name, Param: {}Error message: SemanticError: `Person': Unknown tag

The above exception was the direct cause of the following exception:

RetryError: RetryError[<Future at 0x7ff084431f10 state=finished raised ValueError>]"
}
```

なんとなくQueryの生成で失敗していそうな気がする

In [16]:
# KnowledgeGraphのPromptsの作成
from llama_index.core.prompts.base import PromptTemplate, PromptType

graph_query_synthesis_prompt = PromptTemplate(
    (
        "You are a Graph database expert. You are given a question that requires information retrieval from a Nebula Graph. \n"
        "Generate a precise query that retrieves the relevant information needed to answer the question."
    ),
    prompt_type=PromptType.QUESTION_ANSWER,
)

In [17]:
# KnowledgeGraphQueryEngineの作成
from llama_index.core.query_engine import KnowledgeGraphQueryEngine

query_engine = KnowledgeGraphQueryEngine(
    storage_context=storage_context,
    graph_query_synthesis_prompt=graph_query_synthesis_prompt,
    verbose=True,
)

/tmp/ipykernel_439455/1732950042.py:4: DeprecationWarning: Call to deprecated class KnowledgeGraphQueryEngine. (KnowledgeGraphQueryEngine is deprecated. It is recommended to use the PropertyGraphIndex and associated retrievers instead.) -- Deprecated since version 0.10.53.
  query_engine = KnowledgeGraphQueryEngine(


In [18]:
# 質問の実施

response = query_engine.query(
    "Tell me about Peter Quill?",
)
display(Markdown(f"{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Graph Store Query:
MATCH (p:Person {name: 'John Doe'})-[:FRIENDS_WITH]->(friend)
RETURN friend.name
ERROR:llama_index.graph_stores.nebula.nebula_graph_store:Query failed. Query: MATCH (p:Person {name: 'John Doe'})-[:FRIENDS_WITH]->(friend)
RETURN friend.name, Param: {}Error message: Query failed. Query: MATCH (p:Person {name: 'John Doe'})-[:FRIENDS_WITH]->(friend)
RETURN friend.name, Param: {}Error message: SemanticError: `Person': Unknown tag
ERROR:llama_index.graph_stores.nebula.nebula_graph_store:Query failed. Query: MATCH (p:Person {name: 'John Doe'})-[:FRIENDS_WITH]->(friend)
RETURN friend.name, Param: {}Error message: Query failed. Query: MATCH (p:Person {name: 'John Doe'})-[:FRIENDS_WITH]->(friend)
RETURN friend.name, Param: {}Error message: SemanticError: `Person': Unknown tag
ERROR:llama_index.graph_stores.nebula.nebula_graph_store:Query failed. Query: MATCH (p:Person {name: 'John Doe'})

RetryError: RetryError[<Future at 0x7ff084431f10 state=finished raised ValueError>]

In [43]:
from IPython.display import display, Markdown

graph_query = query_engine.generate_query(
    "Tell me about Peter Quill?",
)

graph_query = graph_query.replace("WHERE", "\n  WHERE").replace("RETURN", "\nRETURN")

display(
    Markdown(
        f"""
```cypher
{graph_query}
```
"""
    )
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



```cypher
Sure, I'd be happy to help! To generate a precise query for Nebula Graph, I need to know the specific question you are trying to answer and the schema of your graph database (i.e., the vertices, edges, and their properties). Could you please provide more details on the question and the structure of your graph?
```


In [47]:
%%ngql
MATCH (p:`entity`)-[e:relationship]->(m:`entity`)
  WHERE p.`entity`.`name` == 'Peter Quill'
RETURN p.`entity`.`name`, e.relationship, m.`entity`.`name`;

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,p.entity.name,e.relationship,m.entity.name


In [45]:
%%ngql
MATCH (p:`entity`)-[e:relationship]->(m:`entity`)
  WHERE p.`entity`.`name` == 'Peter Quill'
RETURN p, e, m;

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,p,e,m


In [46]:
%ng_draw

<class 'pyvis.network.Network'> |N|=0 |E|=0